<a href="https://colab.research.google.com/github/yukinaga/neural_network_on_torus/blob/master/neural_network_on_square.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# neural_network_on_torus
大脳異質を模したニューラルネットワークです。

## Importing packages

In [ ]:
# import numpy as np  # CPU
import cupy as np  # GPU
from PIL import Image, ImageDraw
import IPython.display as disp

## A class of neural network on a torus
**connect():**  
Connect all neurons.  

**initialize_network():**  
Initialize parameters and inhibitory ids.  

**forward():**  
Add a bias to the sum of the product of the input and weight, and process it with a step function.   
Weight and bias are updated everytime this method is called.

In [ ]:
class TorusNetwork():
    def __init__(self, n_h, n_w, n_connect):
        n_neuron = n_h * n_w  # Number of neurons in a network
        self.params = (n_h, n_w, n_neuron, n_connect)

        self.connect_ids = None  # Indices of presynaptic neurons 
        self.w = None  # Weight
        self.b = None  # Bias
        self.y = None  # Output of neurons
        self.proj = None  # Is projection neurons
        self.proj_ids = None  # Indices of projection neurons
        self.proj_to_ids = None  # Indices of destination of projection
        self.inhib = None  # Is inhibitory neurons
        self.desire_ids = None  # Indices of desire neurons
        self.sn_sy = None  # Synapse sensitivity
        self.emotion = 0

    def connect(self, proj_ratio, sigma_inter):
        n_h, n_w, n_neuron, n_connect = self.params
        
        # Random choise of projection neurons
        n_proj= int(proj_ratio * n_neuron)
        rand_ids = np.random.permutation(np.arange(n_neuron))
        self.proj_ids = rand_ids[:n_proj]
        self.proj_to_ids = np.random.permutation(self.proj_ids)
        self.proj = np.zeros(n_neuron, dtype=np.bool)
        self.proj[self.proj_ids] = True

        # X-coordinate of interneurons      
        inter_dist_x = np.random.randn(n_neuron, n_connect) * sigma_inter
        inter_dist_x = np.where(inter_dist_x<0, inter_dist_x-0.5, inter_dist_x+0.5).astype(np.int32) 
        x_connect = np.zeros((n_neuron, n_connect), dtype=np.int32)
        x_connect += np.arange(n_neuron).reshape(-1, 1)
        x_connect %= n_w
        x_connect += inter_dist_x
        x_connect = np.where(x_connect<0, x_connect+n_w, x_connect)
        x_connect = np.where(x_connect>=n_w, x_connect-n_w, x_connect)

        # Y-coordinate of interneurons         
        inter_dist_y = np.random.randn(n_neuron, n_connect) * sigma_inter
        inter_dist_y = np.where(inter_dist_y<0, inter_dist_y-0.5, inter_dist_y+0.5).astype(np.int32)        
        y_connect = np.zeros((n_neuron, n_connect), dtype=np.int32)
        y_connect += np.arange(n_neuron).reshape(-1, 1)
        y_connect //= n_w
        y_connect += inter_dist_y
        y_connect = np.where(y_connect<0, y_connect+n_h, y_connect)
        y_connect = np.where(y_connect>=n_h, y_connect-n_h, y_connect)        

        # Indices of connection
        self.connect_ids = x_connect + n_w * y_connect
        
    def initialize_network(self, inhib_ratio, w_mu, w_sigma):
        n_h, n_w, n_neuron, n_connect = self.params

        # Random choise of inhibitory neurons
        n_inhib = int(inhib_ratio * n_neuron)
        rand_ids = np.random.permutation(np.arange(n_neuron))
        inhib_ids = rand_ids[:n_inhib]
        self.inhib = np.zeros(n_neuron, dtype=np.bool)
        self.inhib[inhib_ids] = True
        
        # Initialize weight and bias
        self.w = np.random.randn(n_neuron, n_connect) * w_sigma + w_mu
        # self.w = np.where(np.isin(self.connect_ids, inhib_ids), -self.w, self.w)
        self.w = np.where(self.inhib[self.connect_ids], -self.w, self.w)
        self.w /= np.sum(self.w, axis=1, keepdims=True)
        self.b = np.zeros(n_neuron)
        
        # Initialize output
        self.y = np.random.randint(0, 2, n_neuron)

    def initialize_desire(self, desire_ratio):
        n_h, n_w, n_neuron, n_connect = self.params

        self.sn_sy = np.zeros((n_neuron, n_connect))

        # Random choise of desire neurons
        n_desire = int(desire_ratio * n_neuron)
        rand_ids = np.random.permutation(np.arange(n_neuron))
        self.desire_ids = rand_ids[:n_desire]

    def forward(self, delta_b, mu_u, excite_ratio, ramda_w, decay_ratio, desire_excite_ratio):
        n_h, n_w, n_neuron, n_connect = self.params
        n_excite = int(excite_ratio * n_neuron)

        # Forward calculation of neurons
        self.y[self.proj_to_ids] = self.y[self.proj_ids]  # Projection
        x = self.y[self.connect_ids]
        u = np.sum(self.w*x, axis=1) - self.b
        larger_ids = np.argpartition(-u, n_excite)[:n_excite]
        self.y[:] = 0
        self.y[larger_ids] = 1
        
        # Homeostasis
        self.b = np.where(self.y, self.b+delta_b, self.b-delta_b)
        self.b -= np.mean(self.b)

        # Desire
        self.sn_sy *= decay_ratio
        self.sn_sy = np.where(self.y.reshape(-1, 1)*x, 1, self.sn_sy)
        n_desire_excite = int(desire_excite_ratio * len(self.desire_ids))
        self.emotion = 0
        # print("Desire sum:", self.y[self.desire_ids].sum(), "n excite:", n_desire_excite)
        if self.y[self.desire_ids].sum() >= n_desire_excite:
            self.emotion = 1
            # print("Emotion!")
        
        # Hebbian learning
        self.w += ramda_w*self.emotion*self.sn_sy*self.y.reshape(-1, 1)*x*np.where(self.w<0, -1, 1)
        self.w /= np.sum(self.w, axis=1, keepdims=True)  # Synaptic scaling

## Settings
Settings of torus neural network.

In [ ]:
n_h = 256  # Height of a plane where neurons are located
n_w = 256  # Width of a plane where neurons are located
n_connect = 64  # Number of presynaptic neurons a neuron has 
tnet = TorusNetwork(n_h, n_w, n_connect)

proj_ratio = 0.5  # Ratio of projection neurons
sigma_inter = 4  # Standard deviation of distance to other neurons
tnet.connect(proj_ratio, sigma_inter)

inhib_ratio = 0.2  # Ratio of interneurons
w_mu = 0.25  # Mean value of weight 
w_sigma = 0.08  # Standard deviation of weight

delta_b = 0.01 # change of bias at every time step
mu_u = -0.12
excite_ratio = 0.1
ramda_w = 0.01  # Hebbian learning ratio

desire_ratio = 0.001
decay_ratio = 0.85
desire_excite_ratio = 0.16

frames = 720  # Frames of the movie

## Temporal change of neurons
The below cell shows the temporal change of 2D map of neurons on a torus.

In [ ]:
tnet.initialize_network(inhib_ratio, w_mu, w_sigma)
tnet.initialize_desire(desire_ratio)

# Color of neuron types
c_proj_exc = np.array([0, 0, 255]).reshape(1, -1)
c_proj_inh = np.array([255, 0, 0]).reshape(1, -1)
c_inter_exc = np.array([30, 144, 255]).reshape(1, -1)
c_inter_inh = np.array([255, 105, 180]).reshape(1, -1)

# Color of neurons
proj = tnet.proj.reshape(-1, 1)
inhib = tnet.inhib.reshape(-1, 1)
c_map = np.zeros((n_h*n_w, 3))
c_map = np.where(proj & ~inhib, c_proj_exc, c_map)
c_map = np.where(proj & inhib, c_proj_inh, c_map)
c_map = np.where(~proj & ~inhib, c_inter_exc, c_map)
c_map = np.where(~proj & inhib, c_inter_inh, c_map)

images = []
n_emo = 0
for i in range(frames):
    tnet.forward(delta_b, mu_u, excite_ratio, ramda_w, decay_ratio, desire_excite_ratio)
    y = tnet.y.reshape(-1, 1)

    n_emo += tnet.emotion
    image = np.full((n_h*n_w, 3), 255*tnet.emotion)  # np.zeros((n_h*n_w, 3))
    image = np.where(y, c_map, image)
    image = image.reshape(n_h, n_w, -1).astype(np.uint8)
    image = Image.fromarray(np.asnumpy(image))
    images.append(image)

images[0].save('tnet_movie.gif',
                   save_all=True, append_images=images[1:], optimize=False, duration=100, loop=0)
with open('tnet_movie.gif','rb') as f:
    disp.display(disp.Image(f.read()))

print("Frames:", frames, "n_emo:", n_emo)